In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/Users/sebastiansoderberg/Documents/CBS/Master/2nd Semester/NLP/NLP Exam/cleaned_data.csv')
print(data.head())

                                         id  \
0  1211a23db42e7dd52a43e564a899ebbf1b2d1251   
1  aff92aa2fc6e294efeeed70e9b0a0aefac32b030   
2  bb7757b7bf3d90ec4f1d1915a4699eecafc25d1a   
3  8f25b2c54cb563b98e1f2619401df7137a3f0c77   
4  daa6ca8fe860fa70bed00e2bfd05ab7414b7cb0e   

                                     cleaned_article  \
0  daily mail reporter published est june updated...   
1  lizzie parry mailonline young cancer victim to...   
2  rebel aided al qaedalinked militant seized con...   
3  cnn two right group launched stinging critique...   
4  cnn puffing electronic cigarette already nono ...   

                                  cleaned_highlights  
0  president barack obamas top national security ...  
1  sophie walton diagnosed bone cancer discoverin...  
2  israeli military closed area around crossing o...  
3  new trial resume kremlin critic aleksei navaln...  
4  government proposes explicitly ban use electro...  


In [4]:
pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 17.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.7/281.7 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 3.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [5]:
from collections import defaultdict, Counter
import nltk
from nltk.util import ngrams
import random

In [9]:
sampled_data = data.sample(n=150, random_state=42)
print(sampled_data.head())

                                              id  \
59770   e44535866d0ffef16786e9ab2dccbf3c4d690778   
21362   2c7fb7b897db7f304961e919cd5ef1a5a93877a4   
127324  edf5aaf6c48a14789eec105b32a5ccb3efe5970a   
140509  47b7bda07fc0f09b7c8f4fe52e8d63ce22d0ee9e   
144297  7a37e591a3250e5791f1238e0092e728decf7100   

                                          cleaned_article  \
59770   gerri peev matt chorley published est may upda...   
21362   cnn five cent chinese yuan coin gave tan yuan ...   
127324  victoria woollaston keeping track polar bear p...   
140509  katie davy published est march updated est mar...   
144297  ted thornhill published est january updated es...   

                                       cleaned_highlights  
59770   poll reveals voter trust current leader le pre...  
21362   tan yuan yuan san francisco ballet principal d...  
127324  researcher tracked polar bear canada using hig...  
140509  celebration underway across u today day extrav...  
144297  jerome hauer

In [10]:
def generate_ngrams(text, n):
    tokens = text.split()
    return list(ngrams(tokens, n))

def build_ngram_model(data, n=3):
    ngram_model = defaultdict(Counter)
    for article in data['cleaned_article']:
        ngrams_list = generate_ngrams(article, n)
        for ngram in ngrams_list:
            prefix = ngram[:-1]
            next_word = ngram[-1]
            ngram_model[prefix][next_word] +=1
    return ngram_model

ngram_model = build_ngram_model(sampled_data, n=3)


In [12]:
def generate_summary(model, start_words, max_length = 80):
    current_words = tuple(start_words.split()[:2])
    summary = list(current_words)

    for _ in range(max_length - len(current_words)):
        if current_words in model:
            next_word_candidates = model[current_words]
            next_word = random.choices(list(next_word_candidates.keys()), weights=next_word_candidates.values())[0]
            summary.append(next_word)
            current_words = tuple(summary[-2:])
        else:
            break
    return ' '.join(summary)

In [13]:
#Exmaple usage
start_words = 'daily mail reporter'
summary = generate_summary(ngram_model, start_words)
print(summary)

daily mail last year drink driving incident one user commented hope doesnt get p stack wd afterwards stevens hit back saying theory false flatly rejected president clinton receives flower girl tuesday upon landing pyongyang north korea trip lot nostalgia pyongyang administration relation north korea much time devoted global philanthropic interest speech clinton never strayed far campaign trail remains one world recognizable statesman clinton jumped back onto world stage tuesday unannounced trip north korea u stronger said john mill chairman shopping


In [16]:
pip install rouge

Note: you may need to restart the kernel to use updated packages.


In [23]:
from rouge import Rouge
def evaluate_summary(reference, generated):
    rouge = Rouge()
    scores = rouge.get_scores(generated, reference, avg=True)
    return scores

# Generate and evaluate summaries for the sampled data
results = []
for index, row in sampled_data.iterrows():
    start_words = ' '.join(row['cleaned_article'].split()[:3])
    generated_summary = generate_summary(ngram_model, start_words)
    reference_summary = row['cleaned_highlights']
    scores = evaluate_summary(reference_summary, generated_summary)
    results.append({
        'id': row['id'],
        'generated_summary': generated_summary,
        'reference_summary': reference_summary,
        'rouge_scores': scores
    })

# Sort results by ROUGE-1 F1 score in descending order
sorted_results = sorted(results, key=lambda x: x['rouge_scores']['rouge-1']['f'], reverse=True)

In [24]:
# Display the top 10 results
top_results = sorted_results[:10]

for result in top_results:
    print(f"Article ID: {result['id']}")
    print(f"Generated Summary: {result['generated_summary']}")
    print(f"Reference Summary: {result['reference_summary']}")
    print(f"ROUGE Scores: {result['rouge_scores']}")
    print("\n")

# Explanation of ROUGE scores
print("Explanation of ROUGE scores:")
print("ROUGE-1, ROUGE-2, and ROUGE-L measure different aspects of overlap between the generated and reference summaries.")
print(" - 'r' (Recall): Measures how much of the reference summary is captured by the generated summary.")
print(" - 'p' (Precision): Measures how much of the generated summary is relevant compared to the reference summary.")
print(" - 'f' (F1 Score): Harmonic mean of precision and recall, providing a balance between the two.")


Article ID: 4275495cbab368496369e875568dad4a839e971b
Generated Summary: ewcom mischief phil get ew learned exclusively kevin hart little fockers signed gueststarring role modern family hell play andre new neighbor phil dunphy happens physician yearold comedian see familiar face begin production abc comedy costarred opposite sofia vergara movie soul plane recently hart hartbeat production notched unexpected victory box office debut standup film laugh pain follows recent national tour run stop hometown philadelphia debuted inside box office top ranking pulled nearly million people pub crawl going weekend boston beer garden
Reference Summary: kevin hart play andre new neighbor phil dunphy hart costarred opposite sofia vergara movie soul plane hart notched unexpected victory box office
ROUGE Scores: {'rouge-1': {'r': 1.0, 'p': 0.26666666666666666, 'f': 0.4210526282548477}, 'rouge-2': {'r': 0.7619047619047619, 'p': 0.20512820512820512, 'f': 0.32323231988980716}, 'rouge-l': {'r': 1.0, 'p': 0